In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_datareader as web
import yfinance as yf
import yahoo_fin
from datetime import datetime
import pytz
import xlwt

In [ ]:
tickers = "ALPA4.SA ^BVSP"

In [ ]:
end_date =  '{}-{}-{}' .format( datetime.now().year,  datetime.now().month,  datetime.now().day)

In [ ]:
yahoo_data = yf.download(tickers=tickers, start="2021-11-30", end=end_date, interval='1d')
intraday_data = yf.download(tickers=tickers, period="1d", interval="1h")

In [ ]:
yahoo_data 

In [ ]:
close = yahoo_data['Close']
volume = yahoo_data['Volume']
high = yahoo_data['High']
low = yahoo_data['Low']
# INTRADAY
intraday_close = intraday_data['Close']
intraday_high = intraday_data['High']
intraday_low = intraday_data['Low']


In [ ]:
intraday_close

In [ ]:
intraday_volume = intraday_data['Volume']
intraday_high

In [ ]:
dados = pd.DataFrame()
dados['Volume'] = volume.iloc[:,-1]
dados['Close'] = close.iloc[:,-1]
dados['Intraday Close'] = intraday_close.iloc[:,-1]
MME3 = close.ewm(span=3).mean()
MME7 = close.ewm(span=7).mean()
dados['MME3'] = MME3.iloc[:,-1]
dados['MME7'] = MME7.iloc[:,-1]
both_close =  pd.concat([close,intraday_close.tail(1)]) 

In [ ]:
def Volume_Projetado(VolumeAtual):
  now = datetime.now()
  BrazilHour = now.hour -3
  BrazilMinute = now.minute
  diffMinutes = 60 - BrazilMinute

  diffClose = (18 - BrazilHour) * 60 - BrazilMinute
  diffOpen = (BrazilHour - 10) * 60 + BrazilMinute

  VolumeFaltante = diffClose*(VolumeAtual/diffOpen)

  VolumeProjetado = VolumeAtual + VolumeFaltante
  return VolumeProjetado

In [ ]:
# DELTA AVAT
# Volume Projetado
last_two = yf.download(tickers=tickers, period='2d', interval='1d')['Volume'].iloc[:-1]
last_two.iloc[:-1] = Volume_Projetado(last_two.iloc[:-1] )
last_two

In [ ]:
### RETORNO CLOSE ANTERIOR E ULTIMO PREÇO
dados['Retorno (Close[1]/Intra)'] = ((dados['Intraday Close'] / dados['Close'] ) -1 ) * 100


In [ ]:
#### DELTA AVAT 1D
# VOLUME PROJETADO / 1D AVAT
delta_avat_1d = (last_two.iloc[0, -1] / last_two.iloc[0, -2]) - 1
dados['Volume Projetado'] = last_two.iloc[-1]
dados['Delta AVAT 1D'] = (dados['Volume Projetado']/dados['Volume']) - 1
dados['Fast AVAT'] = (dados['Delta AVAT 1D'] > 0.5) & (dados['Retorno (Close[1]/Intra)'] < 0.5 )  & (dados['Retorno (Close[1]/Intra)'] > -0.5 )
dados['Fast AVAT'] = dados['Fast AVAT'].astype(int)

In [ ]:
MME3_intraday = both_close.ewm(span=3).mean()
MME7_intraday = both_close.ewm(span=7).mean()

In [ ]:
dados['MME3_intraday'] = MME3_intraday.iloc[-1]
dados['MME7_intraday'] = MME7_intraday.iloc[-1]

In [ ]:
dados['Fast EMA Cross'] = (dados['MME3'] <  dados['MME3_intraday'])  & ( dados['MME3_intraday'] > dados['MME7_intraday']) & (dados['MME3'] < dados['MME7'])
dados['Fast EMA Cross Minus'] =(dados['MME3'] >  dados['MME3_intraday'])  & ( dados['MME3_intraday'] < dados['MME7_intraday']) & (dados['MME3'] > dados['MME7'])


In [ ]:
dados['Fast EMA Cross'] = (dados['MME3'] <  dados['MME3_intraday'])  & ( dados['MME3_intraday'] > dados['MME7_intraday']) & (dados['MME3'] < dados['MME7'])
dados['Fast EMA Cross Minus'] =(dados['MME3'] >  dados['MME3_intraday'])  & ( dados['MME3_intraday'] < dados['MME7_intraday']) & (dados['MME3'] > dados['MME7'])


In [ ]:
MME10_intraday = both_close.ewm(span=10).mean()
MME28_intraday = both_close.ewm(span=28).mean()
MME10 = close.ewm(span=10).mean()
MME28 = close.ewm(span=28).mean()
dados['MME10'] = MME10.iloc[:,-1]
dados['MME28'] = MME28.iloc[:,-1]
dados['MME10_intraday'] = MME10_intraday.iloc[:,-1]
dados['MME28_intraday'] = MME28_intraday.iloc[:,-1]
dados['Slow EMA Cross'] =  (dados['MME10'] <  dados['MME10_intraday'])  & ( dados['MME10_intraday'] > dados['MME28_intraday']) & (dados['MME10'] < dados['MME28'])
dados['Slow EMA Cross Minus'] = (dados['MME10'] >  dados['MME10_intraday'])  & ( dados['MME10_intraday'] < dados['MME28_intraday']) & (dados['MME10'] > dados['MME28'])
dados['Slow EMA Cross'] = dados['Slow EMA Cross'].astype(int)
dados['Slow EMA Cross Minus'] = dados['Slow EMA Cross Minus'].astype(int)*-1

In [ ]:
# 9.1 de Compra
# MME9 (ONTEM) > MME9_anterior (ANTEONTEM)
MME9_anterior_anterior = close.iloc[:,-2].ewm(span=9).mean()
MME9_anterior = close.iloc[:,-1].ewm(span=9).mean()
MME9 = close.ewm(span=9).mean()
# Close de ontem vs close de anteontem
close_anteontem = close.iloc[:, -2]
close_ontem =  close.iloc[:, -1]
# máxima DE ONTEM
high_ontem = high.iloc[:,-1]
# O VALOR INTRADAY MAX DE HOJE > max de ontem
high_agora = intraday_high.iloc[:,-1]
high_agora

In [ ]:
dados['MME9 Anterior Anterior'] = MME9_anterior_anterior.iloc[-1]
dados['MME9 Anterior'] = MME9_anterior.iloc[:-1]
dados['MME9'] = MME9.iloc[:, -1]
dados['Close Anterior'] = close_anteontem
dados['Close'] = close_ontem
dados['High Ontem'] = high_ontem
dados['High Agora'] = high_agora

In [ ]:
# MACD REVERSAL
# MACD1 = MME 26D - MME 12D
MME26 = both_close.ewm(span=26).mean()
MME12 = both_close.ewm(span=12).mean()
MACD1 = MME26 - MME12
# MACD1[1]
MME26_anterior = both_close.iloc[:-1].ewm(span=26).mean()
MME12_anterior = both_close.iloc[:-1].ewm(span=12).mean()
MACD1_anterior = MME26_anterior - MME12_anterior
# MACD SIGNAL = MME9D
MACD_SIGNAL = both_close.ewm(span=9).mean()
MACD_SIGNAL_anterior = both_close.iloc[:-1].ewm(span=9).mean()
# MACD2 = MACD1 - MACD SIGNAL
MACD2_anterior = MACD1_anterior - MACD_SIGNAL_anterior
MACD2 = MACD1 - MACD_SIGNAL
# Se MACD2 t-1 < 0 e MACD2 t > 0  ==>  sinal de reversão de tendência de baixa para alta
# Se MACD2 t-1 > 0 e MACD2t < 0   ==>  sinal de reversão de tendência de alta para baixa
dados['MACD[1]'] = MACD2_anterior.iloc[-1]
dados['MACD'] = MACD2.iloc[-1]

dados['MACD Reversal'] = (dados['MACD[1]'] < 0 ) & (dados['MACD'] > 0 )
dados['MACD Reversal'] = dados['MACD Reversal'].astype(int)

dados['MACD Reversal Minus'] = (dados['MACD[1]'] > 0 ) & (dados['MACD'] < 0 )
dados['MACD Reversal Minus'] = dados['MACD Reversal Minus'].astype(int)*(-1)


In [ ]:

# DIDI POKE
# DDPlus = MA 3D / MA 8D
# DDMinus = MA 20D / MA 8D
DDPlus = both_close.rolling(3).mean() / both_close.rolling(8).mean()
DDMinus = both_close.rolling(20).mean() / both_close.rolling(8).mean()
# ANTERIOR
DDPlus_anterior = both_close.iloc[:-1].rolling(3).mean() / both_close.iloc[:-1].rolling(8).mean()
DDMinus_anterior = both_close.iloc[:-1].rolling(20).mean() / both_close.iloc[:-1].rolling(8).mean()

# DD CROSS
# Se DD Plus é crescente e cruza DD Minus decrescente → 1
# Se DD Minus é crescente e cruza DD Plus decrescente → -1
dados['DDPlus[1]'] = DDPlus_anterior.iloc[-1]
dados['DDPlus'] = DDPlus.iloc[-1]
dados['DDMinus[1]'] = DDMinus_anterior.iloc[-1]
dados['DDMinus'] = DDMinus.iloc[-1]
dados['DDCross'] = (dados['DDPlus[1]'] < dados['DDMinus[1]']) & (dados['DDPlus'] > dados['DDMinus']) & (dados['DDPlus'] > dados['DDPlus[1]']) & (dados['DDMinus'] < dados['DDMinus[1]']  )
dados['DDCross'] = dados['DDCross'].astype(int)
dados['DDCross Minus'] = (dados['DDPlus[1]'] > dados['DDMinus[1]']) & (dados['DDPlus'] < dados['DDMinus']) & (dados['DDPlus'] < dados['DDPlus[1]']) & (dados['DDMinus'] > dados['DDMinus[1]']  )
dados['DDCross Minus'] = dados['DDCross Minus'].astype(int)* (-1)


In [ ]:

# DD POKE
# Se DD Plus é crescente e cruza DD Minus decrescente e ao mesmo tempo DD Pluse DD Minus cruzam o zero → 1
# Se DD Minus é crescente e cruza DD Plus decrescente ao mesmo tempo DD Plus e DD  Minus cruzam o zero → -1
dados['DDPOKE'] = (dados['DDPlus[1]'] < dados['DDMinus[1]']) & (dados['DDPlus'] > dados['DDMinus']) & (dados['DDPlus'] > dados['DDPlus[1]']) & (dados['DDMinus'] < dados['DDMinus[1]']  ) & (dados['DDPlus[1]'] < 0 ) & (dados['DDPlus'] > 0 ) & (dados['DDMinus[1]'] > 0) & (dados['DDMinus'] < 0 )
dados['DDPOKE'] = dados['DDPOKE'].astype(int)
dados['DDPOKE Minus'] = (dados['DDPlus[1]'] > dados['DDMinus[1]']) & (dados['DDPlus'] < dados['DDMinus']) & (dados['DDPlus'] > dados['DDPlus[1]']) & (dados['DDMinus'] > dados['DDMinus[1]']  ) & (dados['DDPlus[1]'] > 0 ) & (dados['DDPlus'] < 0 ) & (dados['DDMinus[1]'] < 0) & (dados['DDMinus'] > 0 ) 
dados['DDPOKE Minus'] = dados['DDPOKE Minus'].astype(int)* (-1)


In [ ]:
# HURST EXPONENT

from numpy import log, polyfit, var, subtract 

def hurst(ts, n): 
    lags = range(2, n)
    variances = []
    for lag in lags: 
        price_diff = subtract(ts[lag:], ts[:-lag])
        variances.append(var(price_diff))
    m = polyfit(log(lags), log(variances), 1)
    hurst_val = m[0] / 2.0
    return hurst_val

In [ ]:
teste = hurst(dados['Close'],25)

In [ ]:
# 90D
resistencia_90D =  high.iloc[-90:-1].max()
suporte_90D = low.iloc[-90:-1].min()

dados['Breakout 90D'] = ((intraday_high.max() > resistencia_90D) & (high.iloc[-1] < resistencia_90D) ).astype(int) * 0.5
dados['Breakdown 90D'] = ( (intraday_low.min() < suporte_90D) & (low.iloc[-1] > suporte_90D)   ).astype(int) * (-0.5)


# 180D
resistencia_180D =  high.iloc[-180:-1].max()
suporte_180D = low.iloc[-180:-1].min()

dados['Breakout 180D'] = ((intraday_high.max() > resistencia_180D) & (high.iloc[-1] < resistencia_180D) ).astype(int) * 0.5
dados['Breakdown 180D'] = ( (intraday_low.min() < suporte_180D) & (low.iloc[-1] > suporte_180D)   ).astype(int) * (-0.5)

# 260D

resistencia = high.iloc[-260:-1].max()
suporte = low.iloc[-260:-1].min()

dados['Breakout 260D'] = (( high.iloc[-1] < resistencia ) & (intraday_high.max() >= resistencia )).astype(int) 
dados['Breakdown 260D'] = ( ( low.iloc[-1] > suporte ) & (intraday_low.min() <= suporte )).astype(int) * (-1)


#BREAKOUT AND BREAKDOWN

dados['Breakout Signal'] = dados['Breakout 90D'] + dados['Breakdown 90D'] + dados['Breakout 180D'] + dados['Breakdown 180D'] + dados['Breakout 260D'] + dados['Breakdown 260D']


In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


In [ ]:
# somar dados['Fast AVAT']  depois
dados['SCORE'] = dados['Fast EMA Cross'] +  dados['Fast EMA Cross Minus'] +  dados['Slow EMA Cross'] +  dados['Slow EMA Cross Minus'] + dados['MACD Reversal'] +  dados['MACD Reversal Minus'] +  dados['Breakout Signal']


In [ ]:
dados.sort_values( by=['SCORE'], ascending=False)

In [ ]:
dados.to_excel('DestaquesMidLarge.xls')